In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from collections import Counter
from pathlib import Path

In [ ]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [ ]:
# Load the data
file_path = Path('/kaggle/input/loanstats/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# Convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100

# Convert the target column values to low_risk and high_risk based on their values
df['loan_status'] = df['loan_status'].replace({
    'Current': 'low_risk',
    'Late (31-120 days)': 'high_risk',
    'Late (16-30 days)': 'high_risk',
    'Default': 'high_risk',
    'In Grace Period': 'high_risk'
})

df.reset_index(inplace=True, drop=True)

# Display the first 5 rows
print(df.head())

In [ ]:
df.columns

In [ ]:
# Create our features
#df contains columns with all string values
training_df = pd.get_dummies(df, columns=["home_ownership","verification_status","issue_d",
                                "pymnt_plan" , "hardship_flag", "debt_settlement_flag",
                                          "initial_list_status", "next_pymnt_d", 
                                          "application_type"])

X = training_df.drop(columns="loan_status")

# Create our target
y =  df["loan_status"]

In [ ]:
X.describe()

In [ ]:
# Check the balance of our target values
y.value_counts()

In [ ]:
# Resampling with SMOTE
smote = SMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split the resampled data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=1, stratify=y_resampled)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Train Balanced Random Forest
rand_forest_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rand_forest_model.fit(X_train_scaled, y_train)

# Predict and evaluate
predictions = rand_forest_model.predict(X_test_scaled)
accurate_score = accuracy_score(y_test, predictions)
print(f"Balanced Random Forest Accuracy: {accurate_score}")

# Confusion Matrix
con_mat = confusion_matrix(y_test, predictions)
con_mat_df = pd.DataFrame(con_mat, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"])
print(con_mat_df)

# Classification Report
print(classification_report_imbalanced(y_test, predictions))

# Feature Importances
feature_importances = rand_forest_model.feature_importances_
feature_names = X.columns
features_importance = sorted(zip(feature_names, feature_importances), key=lambda x: x[1], reverse=True)
for feature, importance in features_importance:
    print(f"{feature}: ({importance})")


In [ ]:
# Train Easy Ensemble AdaBoost
ensemble_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ensemble_model.fit(X_train_scaled, y_train)

# Predict and evaluate
y_predict = ensemble_model.predict(X_test_scaled)
accurate_score = accuracy_score(y_test, y_predict)
print(f"Easy Ensemble AdaBoost Accuracy: {accurate_score}")

# Confusion Matrix
con_mat = confusion_matrix(y_test, y_predict)
con_mat_df = pd.DataFrame(con_mat, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"])
print(con_mat_df)

# Classification Report
print(classification_report_imbalanced(y_test, y_predict))
